In [2]:
from PIL import Image
import numpy as np
import os

In [3]:
def create_watermark(width, height):
    """
    Generates a simple binary watermark pattern (e.g., a checkerboard).
    This pattern will be embedded into the image.

    Args:
        width (int): The width of the watermark.
        height (int): The height of the watermark.

    Returns:
        numpy.ndarray: A 2D array of 0s and 1s representing the watermark.
    """
    watermark = np.zeros((height, width), dtype=np.uint8)
    for y in range(height):
        for x in range(width):
            if (x + y) % 2 == 0:
                watermark[y, x] = 1
    return watermark

In [5]:
def embed_watermark(image_path, watermark):
    """
    Embeds the fragile watermark into the least significant bit (LSB) of the image's pixels.

    Args:
        image_path (str): The path to the input image file.
        watermark (numpy.ndarray): The watermark pattern to embed.

    Returns:
        PIL.Image.Image: The watermarked image object.
    """
    try:
        # Open the original image and convert it to a NumPy array for manipulation
        original_image = Image.open(r"C:\Users\rehem\OneDrive\Desktop\school\4th year\ICS Project II\models\Uncle_SAM.jpeg").convert('RGB')
        image_array = np.array(original_image, dtype=np.uint8)

        h, w, _ = image_array.shape
        wm_h, wm_w = watermark.shape

        # Ensure the watermark can fit within the image dimensions
        if wm_h > h or wm_w > w:
            raise ValueError("Watermark dimensions are larger than the image dimensions.")

        print(f"Embedding a {wm_w}x{wm_h} watermark into a {w}x{h} image.")

        # Iterate over the region of the image where the watermark will be placed
        for y in range(wm_h):
            for x in range(wm_w):
                # Get the pixel's RGB values
                pixel = image_array[y, x]
                watermark_bit = watermark[y, x]

                # --- LSB Manipulation ---
                # We will alter the LSB of the BLUE channel (index 2)
                # Read the current blue value
                blue_value = pixel[2]

                # If the watermark bit is 1 and the blue value's LSB is 0, add 1.
                # If the watermark bit is 0 and the blue value's LSB is 1, subtract 1.
                # This ensures the LSB matches the watermark bit.
                if watermark_bit == 1 and blue_value % 2 == 0:
                    pixel[2] += 1
                elif watermark_bit == 0 and blue_value % 2 != 0:
                    pixel[2] -= 1

        # Convert the modified NumPy array back to an image
        watermarked_image = Image.fromarray(image_array)
        print("Watermark embedded successfully.")
        return watermarked_image

    except FileNotFoundError:
        print(f"Error: The file at {image_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [6]:
def verify_watermark(image_path, original_watermark):
    """
    Extracts the LSB data from an image and compares it to the original watermark.

    Args:
        image_path (str): The path to the (potentially watermarked) image file.
        original_watermark (numpy.ndarray): The original watermark pattern to check against.

    Returns:
        bool: True if the watermark is intact, False otherwise.
    """
    try:
        # Open the image and convert it to a NumPy array
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image, dtype=np.uint8)

        wm_h, wm_w = original_watermark.shape

        # Extract the LSBs from the same location
        extracted_bits = np.zeros_like(original_watermark)

        for y in range(wm_h):
            for x in range(wm_w):
                # We only need the LSB of the blue channel
                blue_value = image_array[y, x][2]
                extracted_bits[y, x] = blue_value % 2

        # Compare the extracted bits with the original watermark
        if np.array_equal(extracted_bits, original_watermark):
            print("Verification successful: Image integrity is intact.")
            return True
        else:
            # Calculate the percentage of corruption for more detailed feedback
            mismatches = np.sum(extracted_bits != original_watermark)
            total_bits = wm_h * wm_w
            corruption = (mismatches / total_bits) * 100
            print(f"Verification FAILED: Image has been altered! Corruption: {corruption:.2f}%")
            return False

    except FileNotFoundError:
        print(f"Error: The file at {image_path} was not found.")
        return False
    except Exception as e:
        print(f"An error occurred during verification: {e}")
        return False

In [7]:
def verify_watermark(image_path, original_watermark):
    """
    Extracts the LSB data from an image and compares it to the original watermark.

    Args:
        image_path (str): The path to the (potentially watermarked) image file.
        original_watermark (numpy.ndarray): The original watermark pattern to check against.

    Returns:
        bool: True if the watermark is intact, False otherwise.
    """
    try:
        # Open the image and convert it to a NumPy array
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image, dtype=np.uint8)

        wm_h, wm_w = original_watermark.shape

        # Extract the LSBs from the same location
        extracted_bits = np.zeros_like(original_watermark)

        for y in range(wm_h):
            for x in range(wm_w):
                # We only need the LSB of the blue channel
                blue_value = image_array[y, x][2]
                extracted_bits[y, x] = blue_value % 2

        # Compare the extracted bits with the original watermark
        if np.array_equal(extracted_bits, original_watermark):
            print("Verification successful: Image integrity is intact.")
            return True
        else:
            # Calculate the percentage of corruption for more detailed feedback
            mismatches = np.sum(extracted_bits != original_watermark)
            total_bits = wm_h * wm_w
            corruption = (mismatches / total_bits) * 100
            print(f"Verification FAILED: Image has been altered! Corruption: {corruption:.2f}%")
            return False

    except FileNotFoundError:
        print(f"Error: The file at {image_path} was not found.")
        return False
    except Exception as e:
        print(f"An error occurred during verification: {e}")
        return False

In [13]:
input_image_path = "Uncle_SAM.jpeg" 
watermarked_image_path = 'watermarked_image.png'
tampered_image_path = 'tampered_image.png'

In [14]:
watermark_pattern = create_watermark(100, 100)
print("Generated a 100x100 watermark pattern.")

Generated a 100x100 watermark pattern.


In [15]:
if not os.path.exists(input_image_path):
    print("\n" + "="*50)
    print(f"ERROR: File '{input_image_path}' not found!")
    print("Please upload your image to the Colab session and make sure the filename matches.")
    print("="*50)
else:
    print("\n--- Embedding Process ---")
    # Embed the watermark into the uploaded image
    watermarked_img = embed_watermark(input_image_path, watermark_pattern)

    if watermarked_img:
        # Save the watermarked image
        watermarked_img.save(watermarked_image_path)
        print(f"Watermarked image saved as '{watermarked_image_path}'")

        # Display the images in Colab (optional, but helpful for visualization)
        try:
            from google.colab.patches import cv2_imshow
            import cv2
            print("\nOriginal Image:")
            cv2_imshow(cv2.imread(input_image_path))
            print("Watermarked Image (visually identical):")
            cv2_imshow(cv2.imread(watermarked_image_path))
        except ImportError:
            print("\n(To display images, run this in a Google Colab environment)")


        print("\n--- Verification on Untouched Watermarked Image ---")
        # Verify the newly created watermarked image (should pass)
        verify_watermark(watermarked_image_path, watermark_pattern)


        print("\n--- Simulating Image Tampering ---")
        # Now, let's alter the watermarked image to test the fragility.
        # We will crop the image by a few pixels.
        with Image.open(watermarked_image_path) as img:
            # Get dimensions
            width, height = img.size
            # Crop: left, top, right, bottom
            # Cropping just 1 pixel off the right side will corrupt the watermark area
            tampered_img = img.crop((0, 0, width - 1, height))
            tampered_img.save(tampered_image_path)
            print(f"Created a tampered version by cropping and saved as '{tampered_image_path}'")

        print("\n--- Verification on Tampered Image ---")
        # Try to verify the tampered image (should fail)
        verify_watermark(tampered_image_path, watermark_pattern)


--- Embedding Process ---
Embedding a 100x100 watermark into a 714x960 image.
Watermark embedded successfully.
Watermarked image saved as 'watermarked_image.png'

(To display images, run this in a Google Colab environment)

--- Verification on Untouched Watermarked Image ---
Verification successful: Image integrity is intact.

--- Simulating Image Tampering ---
Created a tampered version by cropping and saved as 'tampered_image.png'

--- Verification on Tampered Image ---
Verification successful: Image integrity is intact.
